In [69]:
import sys
!{sys.executable} -m pip3 install pandas
!{sys.executable} -m pip3 install numpy
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install sklearn
!{sys.executable} -m pip install xgboost

c:\users\majid\appdata\local\programs\python\python37\python.exe: No module named pip3
c:\users\majid\appdata\local\programs\python\python37\python.exe: No module named pip3


You should consider upgrading via the 'c:\users\majid\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\majid\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\majid\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [70]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import xgboost

import math
pandas.options.mode.chained_assignment = None

##### Read train and test data

In [14]:
train_data = pandas.read_csv('dataset/data_train.csv', header=None, names=['feature'+str(i) for i in range(16)]+['label'])
test_data = pandas.read_csv('dataset/data_test.csv', header=None, names=['feature'+str(i) for i in range(16)]+['label'])
train_data.head()

,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,label
0,47,100,27,81,57,37,26,0,0,23,56,53,100,90,40,98,8
1,0,89,27,100,42,75,29,45,15,15,37,0,69,2,100,6,2
2,0,57,31,68,72,90,100,100,76,75,50,51,28,25,16,0,1
3,0,100,7,92,5,68,19,45,86,34,100,45,74,23,67,0,4
4,0,67,49,83,100,100,81,80,60,60,40,40,33,20,47,0,1


#### Part A: Train a random forecst classifier 

In [38]:
rf_clf = RandomForestClassifier(n_estimators=15, max_features=3, max_depth=3, random_state=0)  # random forest with 15 tree
features = train_data.columns
rf_clf.fit(train_data[features[:-1]], train_data[features[-1]])  # train random forest

RandomForestClassifier(max_depth=3, max_features=3, n_estimators=15,
                       random_state=0)

##### predict labels for test data and evaluate results

In [63]:
features = test_data.columns
rf_predicted = rf_clf.predict(test_data[features[:-1]])  # predict labels for test data

confusion_matrix = metrics.confusion_matrix(test_data['label'], rf_predicted)  # calculate confusion matrix
f1_score = metrics.f1_score(test_data['label'], rf_predicted, average='weighted')
accuracy = metrics.accuracy_score(test_data['label'], rf_predicted)

print('f1-score: {:.3f}'.format(f1_score))
print('accuracy: {:.3f}'.format(accuracy))
print('confusion matrix:')
print(confusion_matrix)

f1-score: 0.786
accuracy: 0.791
confusion matrix:
[[315   4   0   0   0   0   1   3  40   0]
 [  0 188 135   1   2   0   0   0   0  38]
 [  0  18 342   1   1   0   0   2   0   0]
 [  0   7   0 328   0   0   0   0   0   1]
 [  0   2   0   1 354   0   0   0   0   7]
 [  0   0   0 116  13 162   0   0   6  38]
 [  3   0   2   3   0   0 297  29   2   0]
 [  0  22  10   3   0   3   3 294   0  29]
 [ 16   0   0   0   0  12   2  22 283   1]
 [  0  52   0  63  13   0   2   0   2 204]]


#### Part B: Train Adaboost classifier

In [64]:
base_classifier = DecisionTreeClassifier(max_depth=3)
adaboost_clf = AdaBoostClassifier(base_estimator=base_classifier, n_estimators=10, random_state=0) # adaboost with 10 decision tree
features = train_data.columns
adaboost_clf.fit(train_data[features[:-1]], train_data[features[-1]])  # train adaboost 

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=3),
                   n_estimators=10, random_state=0)

##### Test and evaluate adabost classifier with 10 decision tree

In [65]:
def test_and_eval_adaboost(adaboost_clf, test_data, show_confusion_matrix):
    """ This function gets a adaboost classifier and test data then predists labels of test data and evaluates results """
    features = test_data.columns
    adaboost_predicted = adaboost_clf.predict(test_data[features[:-1]])  # predict labels for test data

    confusion_matrix = metrics.confusion_matrix(test_data['label'], rf_predicted)  # calculate confusion matrix
    f1_score = metrics.f1_score(test_data['label'], adaboost_predicted, average='weighted')
    accuracy = metrics.accuracy_score(test_data['label'], adaboost_predicted)

    print('   f1-score: {:.3f}'.format(f1_score))
    print('   accuracy: {:.3f}'.format(accuracy))
    if show_confusion_matrix:
        print('   confusion matrix:')
        print(confusion_matrix)

print('AdaBoost with 10 decision tree:')
test_and_eval_adaboost(adaboost_clf, test_data, True)

AdaBoost with 10 decision tree:
   f1-score: 0.654
   accuracy: 0.669
   confusion matrix:
[[315   4   0   0   0   0   1   3  40   0]
 [  0 188 135   1   2   0   0   0   0  38]
 [  0  18 342   1   1   0   0   2   0   0]
 [  0   7   0 328   0   0   0   0   0   1]
 [  0   2   0   1 354   0   0   0   0   7]
 [  0   0   0 116  13 162   0   0   6  38]
 [  3   0   2   3   0   0 297  29   2   0]
 [  0  22  10   3   0   3   3 294   0  29]
 [ 16   0   0   0   0  12   2  22 283   1]
 [  0  52   0  63  13   0   2   0   2 204]]


#### Part C: Repeat AdaBoost classifier with 5, 20 and 50 decision tree

In [66]:
features = train_data.columns
base_classifier = DecisionTreeClassifier(max_depth=3)
for n_estimator in [5, 10, 20, 50]:
    adaboost_clf = AdaBoostClassifier(base_estimator=base_classifier, n_estimators=n_estimator, random_state=0)
    adaboost_clf.fit(train_data[features[:-1]], train_data[features[-1]])  # train adaboost 
    print('AdaBoost with {} decision tree: '.format(n_estimator))
    test_and_eval_adaboost(adaboost_clf, test_data, False)    

AdaBoost with 5 decision tree: 
   f1-score: 0.580
   accuracy: 0.601
AdaBoost with 10 decision tree: 
   f1-score: 0.654
   accuracy: 0.669
AdaBoost with 20 decision tree: 
   f1-score: 0.759
   accuracy: 0.771
AdaBoost with 50 decision tree: 
   f1-score: 0.777
   accuracy: 0.789


#### Part D: Use XGBoost as classifier and find best parameters

###### Split train data to train and validation data to find best parameters

In [72]:
train_for_xgb = train_data.sample(frac=0.80, random_state=159)
val_for_xgb = train_data.drop(train_for_xgb.index)

###### Define different values for XGBoost parameters and find best parameters

In [98]:
PARAMETERS = {"subsample":[0.5, 0.75, 1], "colsample_bytree":[0.3, 0.5, 0.75, 1], "max_depth":[3,8, 12],
              "min_child_weight":[1,5,15], "learning_rate":[0.01, 0.1, 0.2], "n_estimators":[10, 20, 50]}  # different params
features = train_for_xgb.columns
model = xgboost.XGBClassifier(n_jobs=-1, eval_metric='mlogloss')
model_with_greed_search = GridSearchCV(model, param_grid=PARAMETERS, cv=2, scoring="accuracy")
eval_set = [(train_for_xgb[features[:-1]], train_for_xgb['label']), (val_for_xgb[features[:-1]], val_for_xgb['label'])]
model_with_greed_search.fit(train_for_xgb[features[:-1]], train_for_xgb['label'], eval_set=eval_set, 
                            early_stopping_rounds=10, verbose=0)


c:\users\majid\appdata\local\programs\python\python37\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


{'colsample_bytree': 0.5, 'learning_rate': 0.2, 'max_depth': 8, 'min_child_weight': 1, 'n_estimators': 50, 'subsample': 0.5}


###### Print best parameters

In [99]:
print('Best parametes:')
print(model_with_greed_search.best_params_)

Best parametes:
{'colsample_bytree': 0.5, 'learning_rate': 0.2, 'max_depth': 8, 'min_child_weight': 1, 'n_estimators': 50, 'subsample': 0.5}


###### Test XGBoost model for test data and evaluate results

In [102]:
features = test_data.columns
xgb_predictions = model_with_greed_search.predict(test_data[features[:-1]])  # predict labels of test data

confusion_matrix = metrics.confusion_matrix(test_data['label'], xgb_predictions)  # calculate confusion matrix
f1_score = metrics.f1_score(test_data['label'], xgb_predictions, average='weighted')
accuracy = metrics.accuracy_score(test_data['label'], xgb_predictions)

print("XGBoost with best parameters:")
print('   f1-score: {:.3f}'.format(f1_score))
print('   accuracy: {:.3f}'.format(accuracy))
print('   confusion matrix:')
print(confusion_matrix)

XGBoost with best parameters:
   f1-score: 0.967
   accuracy: 0.967
   confusion matrix:
[[342   0   0   0   0   0   0   0  21   0]
 [  0 341  20   1   0   1   0   1   0   0]
 [  0   2 361   1   0   0   0   0   0   0]
 [  0   3   0 332   0   0   0   0   0   1]
 [  0   0   0   0 362   1   1   0   0   0]
 [  0   0   0   8   0 322   0   0   2   3]
 [  0   0   0   0   0   0 334   0   2   0]
 [  0  27   2   0   0   0   0 329   0   6]
 [  2   0   0   0   0   1   0   0 333   0]
 [  0   3   0   2   0   0   0   2   1 328]]
